#### 检验API可用性

In [15]:
import json, urllib.request

url = "https://raw.githubusercontent.com/AlexGustafsson/systembolaget-api-data/main/data/assortment.json"

with urllib.request.urlopen(url) as r:
    data = json.loads(r.read().decode("utf-8"))

print("num_items:", len(data))
print("keys_of_first_item:", sorted(data[0].keys()))


num_items: 9990
keys_of_first_item: ['alcoholPercentage', 'assortment', 'assortmentText', 'bottleText', 'category', 'categoryLevel1', 'categoryLevel2', 'categoryLevel3', 'categoryLevel4', 'color', 'country', 'customCategoryTitle', 'dishPoints', 'ethicalLabel', 'grapes', 'hasCasqueTaste', 'imageModules', 'images', 'isBsAssortment', 'isClimateSmartPackaging', 'isCompletelyOutOfStock', 'isDiscontinued', 'isEthical', 'isFsAssortment', 'isFsTsAssortment', 'isKosher', 'isManufacturingCountry', 'isNews', 'isOrganic', 'isPaAssortment', 'isRecommendedByTasteProfile', 'isRegionalRestricted', 'isSupplierTemporaryNotAvailable', 'isSustainableChoice', 'isTemporaryOutOfStock', 'isTsAssortment', 'isTsLsAssortment', 'isTseAssortment', 'isTssAssortment', 'isTstAssortment', 'isTsvAssortment', 'isWebLaunch', 'originLevel1', 'originLevel2', 'otherSelections', 'packagingCO2ImpactLevel', 'packagingLevel1', 'price', 'producerName', 'productId', 'productLaunchDate', 'productNameBold', 'productNameThin', 'prod

#### 预处理、过滤、分块

In [16]:
import json
import pandas as pd
import urllib.request

URL = "https://raw.githubusercontent.com/AlexGustafsson/systembolaget-api-data/main/data/assortment.json"

with urllib.request.urlopen(URL) as r:
    data = json.loads(r.read().decode("utf-8"))

df = pd.DataFrame(data)
df.shape


(9990, 78)

In [17]:
# 1) 去掉没有 price 的行
df2 = df[df["price"].notna()].copy()

# 2) 过滤高价酒：去掉价格最高的 10%
price_cap = df2["price"].quantile(0.90)
df2 = df2[df2["price"] <= price_cap].copy()

# 3) 四分位分箱：4 个等级，各 25%
df2["price_class"] = pd.qcut(df2["price"], q=4, labels=[0, 1, 2, 3])

price_cap, df2.shape

(np.float64(1260.1000000000004), (8991, 79))

In [18]:
df2["price_class"].value_counts().sort_index()

price_class
0    2257
1    2403
2    2129
3    2202
Name: count, dtype: int64

#### 使用现实中酒瓶标签上几乎一定能看到的特征集，而且在当前数据里也存在。

In [19]:
ui_features = [
    "country",
    "categoryLevel1",  # Wine/Beer/Spirits…
    "categoryLevel2",
    "alcoholPercentage",
    "volume",
    "vintage",
    "isOrganic",
]

# 只保留存在的列，避免报错
ui_features = [c for c in ui_features if c in df2.columns]

X = df2[ui_features].copy()
y = df2["price_class"].astype(int).copy()

X.head(), y.value_counts().sort_index()


(          country categoryLevel1   categoryLevel2  alcoholPercentage  volume  \
 0         Spanien            Vin         Rött vin               14.5     750   
 1       Frankrike            Vin  Mousserande vin               12.0     750   
 3       Sydafrika   Alkoholfritt             Rosé                0.5     250   
 4  Storbritannien          Sprit           Whisky               46.0     700   
 6         Italien            Vin          Vinlåda               13.0    2250   
 
   vintage  isOrganic  
 0    2023      False  
 1    None      False  
 3    None      False  
 4    None      False  
 6    None      False  ,
 price_class
 0    2257
 1    2403
 2    2129
 3    2202
 Name: count, dtype: int64)

#### 划分数值/类别列，填充缺失等

In [20]:
# 数值列 / 类别列（按 dtype 自动分）
num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

# 填充缺失
for c in cat_cols:
    X[c] = X[c].astype("object").fillna("Unknown")
# 数值缺失 → 用中位数
for c in num_cols:
    X[c] = X[c].fillna(X[c].median())

# 让 isOrganic 变成明确的 0/1（有些数据可能是 True/False/None）
if "isOrganic" in X.columns:
    X["isOrganic"] = X["isOrganic"].astype("bool").astype(int)

X.isna().mean().sort_values(ascending=False).head(10)

C:\Users\cheny\AppData\Local\Temp\ipykernel_18324\1719959949.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[c] = X[c].astype("object").fillna("Unknown")


country              0.0
categoryLevel1       0.0
categoryLevel2       0.0
alcoholPercentage    0.0
volume               0.0
vintage              0.0
isOrganic            0.0
dtype: float64

#### One-hot + XGBClassifier

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# preprocess: numeric median + categorical most_frequent + onehot
preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
        ]), num_cols),
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]), cat_cols),
    ]
)

xgb = XGBClassifier(
    n_estimators=600,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="multi:softprob",
    num_class=4,
    eval_metric="mlogloss",
    tree_method="hist",
    random_state=42
)

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", xgb),
])

model.fit(X_train, y_train)
pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))


Accuracy: 0.6475819899944414


#### one-hot 后的特征维度

In [23]:
model.named_steps["preprocess"].fit(X_train)
Xt = model.named_steps["preprocess"].transform(X_train)
Xt.shape

(7192, 178)

#### 保存模型

In [24]:
import os, joblib

os.makedirs("artifacts", exist_ok=True)

artifact = {
    "model": model,                 # 这个是 OneHot + XGB 的 Pipeline
    "ui_features": ui_features,     # Streamlit 输入字段顺序
    "class_names": {
        0: "Q1 (cheapest)",
        1: "Q2",
        2: "Q3",
        3: "Q4 (most expensive within filtered range)"
    }
}

joblib.dump(artifact, "artifacts/xgb_price_class_model.joblib")
"saved: artifacts/xgb_price_class_model.joblib"


'saved: artifacts/xgb_price_class_model.joblib'

In [25]:
import os
os.path.exists("artifacts/xgb_price_class_model.joblib")


True

#### UI显示具体的价格区间：用 df2（过滤后的数据） 来算边界，追加保存到joblib

In [27]:
# 用 df2 的 price 来计算四分位边界（对应你定义 price_class 的方式）
q0 = float(df2["price"].min())
q25 = float(df2["price"].quantile(0.25))
q50 = float(df2["price"].quantile(0.50))
q75 = float(df2["price"].quantile(0.75))
q100 = float(df2["price"].max())

price_bins = [
    (q0, q25),
    (q25, q50),
    (q50, q75),
    (q75, q100),
]
price_bins
import joblib

path = "artifacts/xgb_price_class_model.joblib"
artifact = joblib.load(path)

artifact["price_bins"] = price_bins  # 4 个区间，每个是 (low, high)

joblib.dump(artifact, path)
"updated: artifacts/xgb_price_class_model.joblib"

'updated: artifacts/xgb_price_class_model.joblib'